# AOC 2020

## Day 9
### Part 1

In [ ]:
with open('input09.txt') as fin:
    lines = [int(line.strip()) for line in fin.readlines()]

In [ ]:
def get_sums(numbers):
    return set(nb1 + nb2 for nb1 in set(numbers) for nb2 in set(numbers) if nb1 != nb2)

def check(numbers):
    pre = 25
    for i in range(pre, len(numbers)):
        if numbers[i] not in get_sums(numbers[i-pre:i]):
            return numbers[i]

In [ ]:
check(lines)

### Part 2

In [ ]:
target = check(lines)
max_idx = lines.index(target)

In [ ]:
def find_weakness(numbers):
    for i in range(max_idx):
        for j in range(i, max_idx):
            if sum(numbers[i:j]) == target:
                return min(numbers[i:j]) + max(numbers[i:j])

In [ ]:
find_weakness(lines)

## Day 8
### Part 1

In [ ]:
with open('input08.txt') as fin:
    lines = [line.strip() for line in fin.readlines()]

In [ ]:
def execute(line):
    cmd, arg = line.split()
    if cmd == 'nop':
        return 1, 0
    arg = int(arg)
    if cmd == 'acc':
        return 1, arg
    if cmd == 'jmp':
        return arg, 0
    
def run(prog):
    idx = 0
    acc = 0
    cache = {0}
    while True:
        off, inc = execute(prog[idx])
        acc += inc
        idx += off
        if idx in cache:
            return False, acc
        if idx >= len(prog):
            return True, acc
        cache.add(idx)
        
_, ans = run(lines)
print(ans)

### Part2

In [ ]:
def fix(prog):
    for i in range(len(prog)):
        if prog[i][:3] == 'acc':
            continue
        fixed = prog.copy()
        if fixed[i][:3] == 'nop':
            fixed[i] = fixed[i].replace('nop', 'jmp')
        else:
            fixed[i] = fixed[i].replace('jmp', 'nop')
        ok, ans = run(fixed)
        if ok:
            print(ans)
            break
            
fix(lines)

## Day 7
### Part 1

In [ ]:
with open('input07.txt') as fin:
    lines = fin.readlines()

In [ ]:
import re
from collections import defaultdict

constraints = {}
reverse = defaultdict(set)

for line in lines:
    bag, content = line.strip().split(' bags contain ')

    constraints[bag] = []

    if not re.match(r"^no other", content):
        content = content.split(", ")
        for item in content:
            match = re.match(r"^(\d) (\w+ \w+) bags?\.?", item)
            if match:
                constraints[bag].append((int(match.group(1)), match.group(2)))
                reverse[match.group(2)].add(bag)

In [ ]:
targets = reverse["shiny gold"].copy()

while True:
    new_targets = set()
    for bag in targets:
        new_targets.update(reverse[bag])
    if new_targets - targets:
        targets.update(new_targets)
    else:
        break
len(targets)

### Part 2

In [ ]:
def recurse(bag):
    if constraints[bag]:
        count = 0
        for qty, subbag in constraints[bag]:
            count += qty*(1+recurse(subbag))
        return count
    else:
        return 0
    
recurse("shiny gold")

## Day 6
### Part 1

In [ ]:
with open('input06.txt') as fin:
    lines = fin.readlines()
    groups = ''.join(lines).split('\n\n')

In [ ]:
def parse_group(group):
    return set("".join(group.split('\n')))

print(sum(len(parse_group(group)) for group in groups))

### Part 2

In [ ]:
def parse_group_p2(group):
    inter_set = parse_group(group)
    for subgroup in group.split('\n'):
        inter_set &= set(subgroup)
    return inter_set

In [ ]:
print(sum(len(parse_group_p2(group)) for group in groups))

## Day 5
### Part 1

In [ ]:
with open('input05.txt') as fin:
    lines = fin.readlines()
    codes = [c.strip() for c in lines]

In [ ]:
import re

def idx(code):
    code = re.sub(r"[FL]", '0', code)
    code = re.sub(r"[BR]", '1', code)
    return int(f"0b{code}", 2)

max(idx(code) for code in codes)

### Part 2

In [ ]:
ids = list(sorted([idx(code) for code in codes]))

for id1, id2 in zip(ids[:-1], ids[1:]):
    if id1 != id2 - 1:
        print(id1 + 1)

## Day 4
### Part 1

In [ ]:
with open('input04.txt') as fin:
    lines = fin.readlines()
    raw_passports = ''.join(lines).split('\n\n')

In [ ]:
import re

def get_passport(raw_line):
    fields = re.split(r"[\s:]", raw_line)
    return {k:v for k,v in zip(fields[:-1:2], fields[1::2])}

In [ ]:
mandatory = {'byr', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid'}
valid = 0
for raw_pass in raw_passports:
    passport = get_passport(raw_pass)
    if set(passport.keys()) >= mandatory:
        valid += 1
print(valid)

### Part 2

In [ ]:
def vyr(s, low, high):
    r = re.match(r"^\d{4}$", s)
    if r:
        y = int(r.group(0))
        return low <= y <= high
    return False

def vhgt(s):
    r = re.match(r"^(\d+)(cm|in)$", s)
    if r:
        h = int(r.group(1))
        u = r.group(2)
        if u == 'cm':
            return 150 <= h <= 193
        elif u == 'in':
            return 59 <= h <= 76
    return False

rules = {
    'byr': lambda s: vyr(s, 1920, 2002),
    'iyr': lambda s: vyr(s, 2010, 2020),
    'eyr': lambda s: vyr(s, 2020, 2030),
    'hgt': vhgt,
    'hcl': lambda s: bool(re.match(r"^#[0-9a-f]{6}$", s)),
    'ecl': lambda s: bool(re.match(r"^(amb|blu|brn|gry|grn|hzl|oth)$", s)),
    'pid': lambda s: bool(re.match(r"^\d{9}$", s)),
}

In [ ]:
mandatory = {'byr', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid'}
valid = 0
for raw_pass in raw_passports:
    passport = get_passport(raw_pass)
    if set(passport.keys()) - {'cid'} == set(rules.keys()) and all(rule(passport[key]) for key, rule in rules.items()):
        valid += 1
print(valid)

## Day 3
### Part 1

In [ ]:
with open('input03.txt') as fin:
    lines = fin.readlines()
    trees = [line.strip() for line in lines]
    height, width = len(trees), len(trees[0])

In [ ]:
def step(x, y, dx, dy):
    return x+dx, y+dy

def run(dx=3, dy=1):
    cx, cy = 0, 0
    nb_trees = 0
    while cy < height:
        if trees[cy][cx] == '#':
            nb_trees += 1
        cx, cy = step(cx, cy, dx, dy)
        cx = cx % width
    return nb_trees

In [ ]:
run()

### Part 2

In [ ]:
slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
count = 1
for dx, dy in slopes:
    count *= run(dx, dy)
print(count)

## Day 2
### Part 1

In [ ]:
import re
from collections import Counter

def parse(line):
    low, high, letter, password = re.split(r"[- ]", line.strip())
    
    low, high, letter = int(low), int(high), letter[:-1]
    
    return low, high, letter, password

def valid(line):
    low, high, letter, password = parse(line)
    count = Counter(password)
    
    return count[letter] >= low and count[letter] <= high

with open('input02.txt') as fin:
    lines = fin.readlines()
    
    print(len([line for line in lines if valid(line)])) 

### Part 2

In [ ]:
def valid_p2(line):
    low, high, letter, password = parse(line)
    
    return (password[low - 1] == letter) != (password[high - 1] == letter)

In [ ]:
with open('input02.txt') as fin:
    lines = fin.readlines()
    
    print(len([line for line in lines if valid_p2(line)])) 

## Day 1
### Part 1

In [ ]:
with open('input01.txt') as fin:
    data = [int(d.strip()) for d in fin.readlines()]

In [ ]:
for a in data:
    for b in data:
        if a+b == 2020:
            print(a*b)

### Part 2

In [ ]:
for a in data:
    for b in data:
        for c in data:
            if a+b+c == 2020:
                print(a*b*c)